In [1]:
import urllib2, json, pickle
import numpy as np
import pandas as pd

In [5]:
response = urllib2.urlopen('https://api.istex.fr/document/?q=(title:%22mental%20rotation%22%20OR%20subject.value:%22mental%20rotation%22%20OR%20keywords:%22mental%20rotation%22%20OR%20abstract:%22mental%20rotation%22)&size=1000&output=id')
data = json.load(response)
subject_MR_ids = np.array(range(len(data['hits'])), dtype=np.object)
for (i, hit) in enumerate(data['hits']):
    subject_MR_ids[i] = hit['id']
subject_MR_ids = subject_MR_ids.tolist()

In [6]:
def get_top_k(k, file_path, article_column):
    df = pd.DataFrame()
    top = np.array(range(k), dtype=np.object)
    for (i, article) in enumerate(np.array(df.from_csv(file_path)[article_column])):
        if i > k-1:
            break
        top[i] = article.split('/')[4]
    return top.tolist()

In [7]:
def get_ids(json_file):
    # takes a json file with list of articles meta data including a field named 'istex_id'
    # retuns numpy array with the istex_id of the articles
    ids = np.arange(len(json_file), dtype=np.object)
    for (i,s) in enumerate(json_file):
        ids[i] = s['istex_id']
    return ids

In [8]:
qbe_top200 = get_top_k(200, 'QbE.csv', 'Articles')

In [9]:
mlt_top200 = get_top_k(200, 'MLT.csv', 'Article')

In [10]:
df = pd.DataFrame()
active_top200 = np.array(df.from_csv('top200_activelearn.csv')['0']).tolist()

In [11]:
def find_intersection(list_a, list_b):
    return list(set(list_a) & set(list_b))
a = ['hussein', 'tayseer', 'al-natsheh']
b = ['loay', 'hussein', 'al-natsheh']
find_intersection(a, b)

['al-natsheh', 'hussein']

In [12]:
intersection_qbe = find_intersection(subject_MR_ids, qbe_top200)
intersection_mlt = find_intersection(subject_MR_ids, mlt_top200)
find_intersection(subject_MR_ids, active_top200)

['823F414AA910D7417970A91A4840656D09B1ADE1',
 'C99FC1EB83D9CAE7318BF085A3F3B8CE83AA1538']

In [13]:
len(df.from_csv('MLT.csv')['Article'])

391

In [14]:
len(df.from_csv('QbE.csv')['Articles'])

959

In [15]:
find_intersection(get_top_k(391, 'MLT.csv', 'Article'), subject_MR_ids)

[u'21DF3DF37766A07B3D9C35A597D9125DFEF6B2D9']

In [16]:
find_intersection(get_top_k(959, 'QbE.csv', 'Articles'), subject_MR_ids)

[u'64380259D82AB564EF2E11A552ADA86BA0CB2BF2']

In [40]:
def eval_meta(top):
    matches = find_intersection(top, subject_MR_ids)
    print 'count of matches: ',len(matches)
    print 'rank of them:'
    ranks = []
    for (i, r) in enumerate(top):
        if r in matches:
            print i
            ranks.append(i)
    return matches, ranks
    
matches_active, ranks_active = eval_meta(np.array(df.from_csv('top959_activelearn.csv')['0']).tolist())

count of matches:  6
rank of them:
23
81
226
566
698
928


In [41]:
matches_qbe, ranks_qbe = eval_meta(get_top_k(959, 'QbE.csv', 'Articles'))
matches_mlt, ranks_mlt = eval_meta(get_top_k(391, 'MLT.csv', 'Article'))

count of matches:  1
rank of them:
850
count of matches:  1
rank of them:
0


In [42]:
ranks_active

[23, 81, 226, 566, 698, 928]

In [18]:
test = json.load(open('sample_data/MentalRotationInMetaDataIstexWithoutAnnotated.json','r'))
test_ids = get_ids(test)
test_ids = test_ids.tolist()

In [21]:
print 'meta other than MRISTEX: ', len(subject_MR_ids) - len(find_intersection(test_ids, subject_MR_ids))

meta other than MRISTEX:  159


In [22]:
print len(find_intersection(np.array(df.from_csv('top10kwo_activelearn.csv')['0']).tolist(), subject_MR_ids))
find_intersection(np.array(df.from_csv('top10kwo_activelearn.csv')['0']).tolist(), subject_MR_ids)

13


[u'B26B967E3291B9A1D8C2788375EC13EAE37C2172',
 u'C99FC1EB83D9CAE7318BF085A3F3B8CE83AA1538',
 u'50BF4CCD3CB37A8DC0DF733C4EEB9D1E788B2845',
 u'64380259D82AB564EF2E11A552ADA86BA0CB2BF2',
 u'3D2369FDE80305CCF2EA47B90097CC54133139D3',
 u'586B0ECFBC0CCE4C8BCF4769A621A412ED5C432F',
 u'BF41413D6B46AA1792C96B3A15F76E3E5ED75F03',
 u'823F414AA910D7417970A91A4840656D09B1ADE1',
 u'25CF960A29914AAD30CE74DBDABE691930BF5D95',
 u'26951B4999DD3FAA5FAA67241E2DEAE16708E75C',
 u'B819644036DBEE7562203695C995E8AFC829E03F',
 u'FEB63FD713A315826FF6E25BD9689738511E7D88',
 u'58860B6503194D6517FA1A41B215E56E84E352F3']